### RAG Piplelines - Data INgestion to Vector DB Pipleline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\saadf\Desktop\RAG-Youtube_Course\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Reaad all the pdf's insode the directory 

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    #Find all PDF files recursively
    pdf_files = list(pdf_dir.rglob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process.")

    for pdf_file in pdf_files:
        print(f"Processing file: {pdf_file}")
        try:
            loader = PyMuPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata["source_file"] = pdf_file.name
                doc.metadata["source_typr"] = 'pdf'

            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages")
        
        except Exception as e:
            print(f" Error loading: {e}")
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documnets = process_all_pdfs("../data")

Found 2 PDF files to process.
Processing file: ..\data\pdf_files\Al Rouf.pdf
 Loaded 3 pages
Processing file: ..\data\pdf_files\Product & Price Guide.pdf
 Loaded 5 pages

Total documents loaded: 8


In [3]:
all_pdf_documnets

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}, page_content="Al Rouf Lighting Technology Co. Ltd. \nOfficial Company Profile (Internal Document) \nPage 1: About Al Rouf & Our Identity \nOur Company \nEstablished in 2014, Al Rouf Lighting Technology Co. Ltd. is a leading, ISO 9001-2015 QMS \nand ISO 45001:2018 HSE certified company based in the Kingdom of Saudi Arabia. We are a \npremier provider of comprehensive, turnkey lighting solutions. \nOur commitment to quality and innova

In [4]:
### Text splitting get into chunks

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documnets into smaller chunks for better RAG performance"""

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split  {len(documents)}  documents into {len(split_docs)} chunks")

    # show example of a chunk
    if split_docs:
        print("\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
        
    return split_docs

In [5]:
chunks = split_documents(all_pdf_documnets)
chunks

Split  8  documents into 13 chunks

Example chunk:
Content: Al Rouf Lighting Technology Co. Ltd. 
Official Company Profile (Internal Document) 
Page 1: About Al Rouf & Our Identity 
Our Company 
Established in 2014, Al Rouf Lighting Technology Co. Ltd. is a le...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}, page_content="Al Rouf Lighting Technology Co. Ltd. \nOfficial Company Profile (Internal Document) \nPage 1: About Al Rouf & Our Identity \nOur Company \nEstablished in 2014, Al Rouf Lighting Technology Co. Ltd. is a leading, ISO 9001-2015 QMS \nand ISO 45001:2018 HSE certified company based in the Kingdom of Saudi Arabia. We are a \npremier provider of comprehensive, turnkey lighting solutions. \nOur commitment to quality and innova

### Embeddings and vectorStoreDB

In [6]:
!pip install sentence-transformers

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformers"""

    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        Initialize the embedding manager

        Args:
        model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()


    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error Loading model {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for a list of texts

        Args:
        texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Embedding model is not loaded.")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
## initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore 

In [11]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self,collection_name: str = "pdf_documents",persist_directory: str = "../data/vector_store"):
        """Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection 
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistant ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Dict[str, Any]], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
        documents: List of LangChain Documents
        embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of the embeddings")
        
        print(f"Adding {len(documents)} documents to the vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc,embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
           doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
           ids.append(doc_id)

           #prepare metadata
           metadata = dict(doc.metadata)
           metadata['doc_index'] = i
           metadata['content_length'] = len(doc.page_content)
           metadatas.append(metadata)

           # Document text
           documents_text.append(doc.page_content)

           # Embedding
           embeddings_list.append(embedding.tolist())

        # Add to ChromaDB collection
        try:
            self.collection.add(
                ids = ids,
                metadatas = metadatas,
                documents = documents_text,
                embeddings = embeddings_list
            )
            print(f"Successfully added {len(documents)} documents to the vector store.")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error addignn documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [12]:
chunks

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}, page_content="Al Rouf Lighting Technology Co. Ltd. \nOfficial Company Profile (Internal Document) \nPage 1: About Al Rouf & Our Identity \nOur Company \nEstablished in 2014, Al Rouf Lighting Technology Co. Ltd. is a leading, ISO 9001-2015 QMS \nand ISO 45001:2018 HSE certified company based in the Kingdom of Saudi Arabia. We are a \npremier provider of comprehensive, turnkey lighting solutions. \nOur commitment to quality and innova

In [13]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

### Generate the Embeddings

embeddings = embedding_manager.generate_embeddings(texts)


### Store in the vector database
vectorstore.add_documents(chunks, embeddings)


Generating embeddings for 13 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


Generated embeddings with shape: (13, 384)
Adding 13 documents to the vector store...
Successfully added 13 documents to the vector store.
Total documents in collection: 13


### RAG - Retriever Pipeline From VectorStore

In [ ]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the RAG retriever

        Args:
            vector_store: Vector srore containing document embeddings
            embedding_manager: Manager for generating query embeddings
        
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager


    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top documents to retrieve
            score_threshold: Minimum similarity score to consider
        
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score Threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings = [query_embedding.tolist()],
                n_results = top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    #Convert distance to similarity score (ChromsDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            "id": doc_id,
                            "content": document,
                            "metadata": metadata,
                            "similarity_score": similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering")
            else:
                print("No documents found.")

            return retrieved_docs
        
        except Exception as e:
            print(f"Error during retrieval: {e}")
            raise []
        
rag_retriever = RAGRetriever(vectorstore, embedding_manager)

In [15]:
rag_retriever

In [17]:
rag_retriever.retrieve("What Al Rouf?", top_k=3, score_threshold=0.1)

Retrieving documents for query: 'What Al Rouf?'
Top K: 3, Score Threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.40it/s]

Generated embeddings with shape: (1, 384)
Error during retrieval: cannot access local variable 'documents' where it is not associated with a value


TypeError: exceptions must derive from BaseException